In [1]:
#Update your token
STUDENT_TOKEN = 'EUGENIO MARCHIORI'

## ignore this code, just used for submission
import requests
import pprint
import json
import random
import math
from copy import copy, deepcopy

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)

In [2]:
def format_value(value):
  if value == '.':
    return '..'
  else:
    s = str(value)
    if len(s) == 1:
      s = ' ' + s
    return s

def format_h(const, max):
  s = ''
  for i in range(max - len(const)):
    s = s + '   '
  s = s + ' '.join(map(format_value, const))
  return s

def pretty_print(board, h_const, v_const):
  max_h = 1
  for c in h_const:
    max_h = max(max_h, len(c))
  max_v = 1
  for c in v_const:
    max_v = max(max_v, len(c))

  h_offset = ' '.join(map(lambda x: '  ', range(max_h)))
  for i in range(max_v):
    print('    ' + h_offset + ' '.join(map(lambda x: '  ' if i >= len(x) else format_value(x[i]), v_const)))
  for r in range(len(board)):
    row = board[r]
    print((h_offset if r ==0 else format_h(h_const[r-1], max_h)) + ' ' +  ' '.join(map(format_value, row)))

In [4]:
problem = get_problem('kakuro')

pprint.pprint(problem)

board = problem['board']
h_const = problem['h_constraints']
v_const = problem['v_constraints']

print('Input puzzle')
#pretty_print(board, h_const, v_const)

# This does not solve the problem, just makes it the same as the input
solution = board

# We print the solution in a way to make it easy to debug and check
#print('Solution')
#pretty_print(solution, h_const, v_const)

answer = {
    'board': board,
    'h_const': h_const,
    'v_const': v_const,
    'id': problem['id']
}

# The answer is submitted to the server, this is just to check the format (it does not check the solution itself)
# Note: We are doing this just to get used to it in following exercises mostly
submit_answer('kakuro', answer)

{'board': [['X', 'X', 'X', 'X', 'X'],
           ['X', '.', '.', '.', 'X'],
           ['X', '.', '.', '.', 'X'],
           ['X', 'X', '.', '.', '.'],
           ['X', 'X', '.', '.', '.']],
 'h_constraints': [[7], [19], [8], [15]],
 'id': 0,
 'v_constraints': [[5], [12], [22], [10]]}
Input puzzle
Ignore: not checking values yet


In [ ]:
# Representation of the problem

# Initialize the domains. Each empty cell ('.') will have all numbers from 1 to 9 as possible values.
domains = {(row, col): list(range(1, 10)) for row in range(len(board)) for col in range(len(board[0])) if board[row][col] == '.'}

# Initialize the constraints. We will store them in a dictionary where the key is the cell (row, col)
# and the value is a tuple containing the sum constraint for the horizontal and vertical blocks.
constraints = {}

# Populate the constraints dictionary with horizontal constraints
for r, row in enumerate(h_const):
    sum_constraint = row[0]
    cells_in_block = [(r+1, c) for c, value in enumerate(board[r+1]) if value == '.']
    for cell in cells_in_block:
        constraints[cell] = constraints.get(cell, (0, 0))
        constraints[cell] = (sum_constraint, constraints[cell][1])

# Populate the constraints dictionary with vertical constraints
for c, col in enumerate(v_const):
    sum_constraint = col[0]
    cells_in_block = [(r, c+1) for r, value in enumerate(board) if value[c+1] == '.']
    for cell in cells_in_block:
        constraints[cell] = constraints.get(cell, (0, 0))
        constraints[cell] = (constraints[cell][0], sum_constraint)



In [ ]:
# constraint propagation